<a href="https://colab.research.google.com/github/Athu7/Extracting-keywords-from-legal-text-documents-/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import nltk 
import re
import os
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [89]:
train_path = '/content/drive/MyDrive/Lawnics_Assignment_Datascience/Train_docs'
tag_path ='/content/drive/MyDrive/Lawnics_Assignment_Datascience/Train_tags'

In [90]:
numbers = re.compile(r'(\d+)')
def numericalSort(value):
  parts = numbers.split(value)
  parts[1::2] = map(int, parts[1::2])
  return parts

In [91]:
documents_list = sorted(os.listdir(train_path), key = numericalSort)
tags_list = sorted(os.listdir(tag_path), key = numericalSort)

In [92]:
docs = []
tags = []
for i,j in enumerate(documents_list):
  documents_path ='%s/%s' % (train_path,documents_list[i])
  with open(documents_path, 'r', encoding = 'utf-8',errors='ignore') as f:
    document = f.read()
    docs.append(document)
    print(i, j)

0 case_0_statement.txt
1 case_1_statement.txt
2 case_2_statement.txt
3 case_3_statement.txt
4 case_4_statement.txt
5 case_5_statement.txt
6 case_6_statement.txt
7 case_7_statement.txt
8 case_8_statement.txt
9 case_9_statement.txt
10 case_10_statement.txt
11 case_11_statement.txt
12 case_12_statement.txt
13 case_13_statement.txt
14 case_14_statement.txt
15 case_15_statement.txt
16 case_16_statement.txt
17 case_17_statement.txt
18 case_18_statement.txt
19 case_19_statement.txt
20 case_20_statement.txt
21 case_21_statement.txt
22 case_22_statement.txt
23 case_23_statement.txt
24 case_24_statement.txt
25 case_25_statement.txt
26 case_26_statement.txt
27 case_27_statement.txt
28 case_28_statement.txt
29 case_29_statement.txt
30 case_30_statement.txt
31 case_31_statement.txt
32 case_32_statement.txt
33 case_33_statement.txt
34 case_34_statement.txt
35 case_35_statement.txt
36 case_36_statement.txt
37 case_37_statement.txt
38 case_38_statement.txt
39 case_39_statement.txt
40 case_40_statement

In [93]:
for i,j in enumerate(tags_list):
  tags_path = '%s/%s' % (tag_path, tags_list[i])
  with open(tags_path, 'r', encoding = 'utf-8', errors = 'ignore') as t:
    tag = t.read()
    tags.append(tag)

In [94]:
print(len(docs))
print(len(tags))

80
80


In [95]:
# Creating a pandas dataframe 
import pandas as pd
data = {'documents':docs,
        'tags':tags}
df = pd.DataFrame(data)
df.head()

,documents,tags
0,"\n\nKurian Joseph, J.\n\n1. Leave granted in S...","Absence, Access, Accident, Account, Acquisitio..."
1,"\n\nAbhay Manohar Sapre, J.\n\n1. Delay in fil...",Cause of Action
2,"\n\nPinaki Chandra Ghose, J.\n\n1. This crimin...","Abetment, Abetment of Suicide, Absconding, Acc..."
3,\n\n1. This matter is placed before us as a Be...,"Decision, Exemption, Exemption Notification, I..."
4,\n\n1. We have heard learned Counsel for the p...,"Child Labour, Compensation, Fundamental Right,..."


In [96]:
# Cleaning the documents
docs_cleaned =[]
for i in range(0, len(docs)):
  review = re.sub('[^a-zA-Z]', ' ', df['documents'][i])
  review = review.split()
  review = [word for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  docs_cleaned.append(review)

In [97]:
data = {'documents':docs_cleaned,
        'tags':tags}
df = pd.DataFrame(data)
df.head()

,documents,tags
0,Kurian Joseph J Leave granted Special Leave Pe...,"Absence, Access, Accident, Account, Acquisitio..."
1,Abhay Manohar Sapre J Delay filing special lea...,Cause of Action
2,Pinaki Chandra Ghose J This criminal appeal sp...,"Abetment, Abetment of Suicide, Absconding, Acc..."
3,This matter placed us Bench Court differed vie...,"Decision, Exemption, Exemption Notification, I..."
4,We heard learned Counsel parties The prayer ma...,"Child Labour, Compensation, Fundamental Right,..."


#Creating embeddings using distilBERT

In [98]:
!pip install sentence_transformers

     |████████████████████████████████| 78 kB 5.2 MB/s 
     |████████████████████████████████| 3.5 MB 45.8 MB/s 
     |████████████████████████████████| 6.8 MB 41.0 MB/s 
     |████████████████████████████████| 1.2 MB 32.9 MB/s 
     |████████████████████████████████| 67 kB 4.0 MB/s 
     |████████████████████████████████| 895 kB 51.0 MB/s 
     |████████████████████████████████| 596 kB 55.7 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=120999 sha256=ff52a9b49a52058867a7fc80f3af3998e06e15d88c125606b8de1b31ba64902c
  Stored in directory: /root/.cache/pip/wheels/90/f0/bb/ed1add84da70092ea526466eadc2bfb197c4bcb8d4fa5f7bad
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [104]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('distilbert-base-nli-mean-tokens')
doc_embedding = model.encode(docs_cleaned)
tag_embeddings = model.encode(tags)

#**Model Creation**

In [109]:
import tensorflow as tf

In [108]:
from tensorflow.keras.layers import Embedding 
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [110]:
voc_size = 5000
doc_length = 100

In [ ]:
#Creating the model

embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length = doc_length))
model.add(LSTM(100))
model.add((1, activation = 'sigmoid'))
model.compile(loss = "binary_crossentropy", optimizer = 'adam', metrics = ['accuracy'])
print(model.summary())